# Integration Milestone pg: AOS using ComCam, commanding MT 

<div class="alert alert-block alert-warning">
<b>Warning:</b> This notebooks was designed and executed at NTS. It may not work "as-is" in different environments, especially at the summit. It may also stop working at NTS if any of the underlying data changes. In case of problems contact the author for support.
</div>

This notebook is designed to execute Integration Milestone pg (IMpg).

The task consist of the following step:

 1. Enabling the MTAOS with the configuration designed for the test.
 This configuration mainly points the MTAOS to read data from a local butler for wich simulated intra/extra focal data for ComCam was previously ingested.

 2. Send the command `runWEP` specifying the `visitId` and `extraId`, which will cause the MTAOS to process the data in the background using the wavefront estimation pipeline.
 
   If this is the first time data is processed this command can take quite a while to execute (a bit over 10 minutes in most times I executed it).
 
   If the data was already processed, the MTAOS will use the results from the butler and mainly republish the events with the information and configure the internal model to apply the correction.
 
   At this point we don't have a good mechanism that would cause the MTAOS to reprocess a previously processed data.
   One option is to delete the "run" from the butler.
 
   To do this execute the following command in a terminal in nublado:

   ```
   butler prune-collection /project/tribeiro/IMpg/data/im9-ppg-w_2021_20/ mtaos_wep --purge --unstore
   ```

   A couple things to note on the command above. 

    1. The path `/project/tribeiro/IMpg/data/im9-ppg-w_2021_20/` is the location of the butler instance.
       This is enconded in the MTAOS configuration mentioned above.
    2. The string `mtaos_wep` is the name of the run.
       This is also part of the CSC configuration but, in this case, it is the default value used by the MTAOS.
    3. The CSC has access to a shared space (e.g. `/project`) which contains a butler instance with data from the camera and also user-defined butler instances, as the one used in this exercise.

 3. Send the command `runOFC` to the MTAOS.
    This command makes the MTAOS process the wavefront errors generated by `runWEP` and compute the corrections for each component; Camera Hexapod, M2 Hexapod, M2 and M1M3.
 4. Finally, issue the corrections to the components by executing the `issueCorrection` command.
 
Analysis of the outputs is done on a separate notebook (`check_impg_run.ipynb`).


In [ ]:
import sys
import asyncio
import logging

import numpy as np

from lsst.ts import salobj

from lsst.ts.observatory.control.maintel.mtcs import MTCS, MTCSUsages
from lsst.ts.observatory.control.utils.enums import RotType

## Setting up loggers

This next couple cells setup a logger instance for the notebook and set the log level to `DEBUG`. In order to avoid extreme verbosity for some background packages (e.g. matplotlib and others) we set their logging level to `WARNING`.

In [ ]:
stream_handler = logging.StreamHandler(sys.stdout)
logger = logging.getLogger()
logger.addHandler(stream_handler)
logger.level = logging.DEBUG

In [ ]:
logging.basicConfig(format="%(name)s:%(message)s", level=logging.DEBUG)

In [ ]:
logging.getLogger("matplotlib").setLevel(logging.WARNING)

## Setup domain and remotes to drive the tests

For this test we could easily use a remote for the MTAOS alone, especially since we will only do the data analysis in a different notebook. 

Nevertheless, we will use the higher level `MTCS` class so we can get used to operating with it and we can later expand this notebook to accomplish more than simply processing data and applying corrections.

In [ ]:
domain = salobj.Domain()

In the cell bellow we create an instance of the mtcs and set the remote log levels to `CRITICAL`. We do this so we silence some salobj lower level messages when the Remotes cannot keep up with the data traffic. Since the `MTCS` communicates with some pretty chatty components, this is, unfortunately, necessary. 

In [ ]:
mtcs = MTCS(domain, intended_usage=MTCSUsages.All)
mtcs.set_rem_loglevel(logging.CRITICAL)

In [ ]:
await mtcs.start_task

## Test starting

The setup is now completed and we can start the test.

Before moving forward we need to check that the MTAOS is alive, which is done waiting for a heartbeat from the component.

In [ ]:
await mtcs.next_heartbeat("mtaos")

We now set the log level to `DEBUG` so we can inspect any error in the execution.

In [ ]:
await mtcs.rem.mtaos.cmd_setLogLevel.set_start(level=logging.DEBUG)

We need to make sure that the CSC will be enabled with the correct configuration needed to execute the test. 
Therefore, we will first send the CSC to STANDBY and then to ENABLED.

The sequence of commands bellow will make sure that happens regarless of the initial state of the CSC.

In [ ]:
await mtcs.set_state(salobj.State.STANDBY, components=["mtaos"])

In [ ]:
await mtcs.set_state(salobj.State.ENABLED, settings={"mtaos": "impg"}, components=["mtaos"])

Finally we are ready to command the MTAOS to process ComCam data.

The visitId (id of the intra-focal image) and the extraId (id of the extra-focal image) are 4000021706001 and 4000021706003 respectively. Notice in the cell bellow that we only use the last 8 digist of the id.

The problem is that in the current version of the xml (v9.1.1), both parameters are specified as having type `long`, which only yields values between `-2147483648` and `2147483647`.

In order to work around this issue I implemented a `visit_id_offset` configuration parameter in the CSC. The final id is then computed as `visit_id_offset + visitId` and  `visit_id_offset + extraId`.

The configuration designed for this exercives sets `visit_id_offset` to 4000000000000 so we only need to pass `visitId=21706001` and `extraId=21706003` to the CSC.

For future releases we will update the type of these fields to `unsigned long long`, which supports numbers from zero up to `18446744073709551615`.

It is worth noting that the visit id is an integer number constructed from `YYYYMMDDXXXXX`, where `YYYY` is the year, `MM` is the month and `DD` is the day of the observation and `XXXXX` is an incremental sequence number that resets every day. That means a regular `long` type won't be even able to cover the current sequence number, and an `unsigneg long long` will last until year 18446744073, which would be sufficient for ~ 10^10 years of operations.


In [ ]:
ack = await mtcs.rem.mtaos.cmd_runWEP.set_start(
    visitId=21706001,
    extraId=21706003,
    wait_done=True
    )

Now the the WEP finished we can tell the MTAOS to process the data using the OFC to compute the corrections yielded by the wavefront sensor.

In [ ]:
await mtcs.rem.mtaos.cmd_runOFC.set_start()

Finally, we issue the corrections to the components.

In [ ]:
await mtcs.rem.mtaos.cmd_issueCorrection.start()

### Final thoughts

If you want to experiment computing the corrections with different OFC configurations, you can run the command

```
await mtcs.rem.mtaos.cmd_resetCorrection.start()
```

Then rerun `cmd_runWEP` and `cmd_runOFC` with some custom configuration.

## Closing remarks

At NTS we usually leave the system running, so there is no need to shutdown at the end.

The from the run can now be analysed from the EFD.
You need to register the execution times so you can fill in the start/end times on the analysis notebook.